In [1]:
import sys
sys.path.append("../")

import csv
import pandas as pd
import numpy as np
import json
from itertools import repeat, chain
import utils
from utils import pad_with_zeros, zeros

In [2]:
## Update this to wherever your data directory is that contains that downloaded files referenced in the README
## of this repo
data_directory = "data"

## Prep - Scoping to just most populous counties

In [5]:
# Read the recent populations estimates for all counties in the US
population = pd.read_csv(f"{data_directory}/co-est2019-alldata.csv", header=0, encoding='ISO-8859-1')

In [6]:
# Get counties with population greater than 100K - this can be changed and is just to narrow our initial scope of data
pop = population[population['POPESTIMATE2019'] > 100000]

# Filter out entries that are not cities
pop =  pop[pop['CTYNAME'].str.contains('County')]

# Create unique state and place code for merge
pop['STATE_COUNTY'] = pop.apply(lambda x: f"{x['STATE']}:{x['COUNTY']}", axis=1)

# Generate a list of valid states and place
filter_pop = pop['STATE_COUNTY'].to_list()

In [7]:
# Save the unique state and county pairs that we will be querying from the ACS
with open(f'{data_directory}/state_county.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(list(map(lambda x: x.split(":"), filter_pop)))

## Running to Collect ACS Data using Census API

In [8]:
# Lets set up our requests session. You can use teh creaet session utility or just pass the requests object
session = utils.create_session()

states_and_counties_fp = "{}/state_county.csv".format(data_directory)

# Read in the states and places we will be querying
states_and_counties = pd.read_csv(states_and_counties_fp).values

In [ ]:
## These codes and endpoints are from viewing the Census API documentation
## This google doc contains a mapping of these variables to descriptions 
##  - https://docs.google.com/document/d/1QCS26WSUEZu5zacdtFbVvJgc0DD9OVEh02aNuEFgZ78/edit?usp=sharing
## you can also find them on the https://data.census.gov/cedsci/
codes = {'acs/acs5':[ "B01003_001E", "B01002_002E", "B01002_001E", "B02001_002E", "B06009_001E",
                    "B06009_002E", "B06009_003E", "B06009_005E", "B06009_006E"],
        'acs/acs5/profile' : ["DP02_0001E", "DP04_0037E", "DP03_0062E"]} #TODO #All missing "B06011_001E"
acs_endpoints = ['acs/acs5', 'acs/acs5/profile']


df = utils.gather_results(session, acs_endpoints, utils.config, states_and_counties, codes, dfs=[], start=0)


1 15
Status: 0.17%
1 55
Status: 0.35%
1 69
Status: 0.52%
1 73


In [9]:
df.columns

Index(['NAME', 'B01003_001E', 'B01002_002E', 'B01002_001E', 'B02001_002E',
       'B06009_001E', 'B06009_002E', 'B06009_003E', 'B06009_005E',
       'B06009_006E', 'state', 'county', 'tract', 'DP02_0001E', 'DP04_0037E',
       'DP03_0062E'],
      dtype='object')

In [10]:
df.to_csv("census_tracts.csv", index= False)

In [11]:
df.shape

(54450, 16)

## Merging Data by Tract - Start here if already collected ACS data

In [3]:
census_tracts = pd.read_csv("census_tracts.csv")
census_tracts

,NAME,B01003_001E,B01002_002E,B01002_001E,B02001_002E,B06009_001E,B06009_002E,B06009_003E,B06009_005E,B06009_006E,state,county,tract,DP02_0001E,DP04_0037E,DP03_0062E
0,"Census Tract 20, Calhoun County, Alabama",7331,39.2,39.8,6926,5131,387,1758,598,481,1,15,2000,2735,6.0,66217
1,"Census Tract 8, Calhoun County, Alabama",981,39.2,41.7,419,752,204,278,43,35,1,15,800,423,5.1,27656
2,"Census Tract 9, Calhoun County, Alabama",3617,40.9,44.3,2476,2610,266,638,459,448,1,15,900,1528,6.4,50810
3,"Census Tract 14, Calhoun County, Alabama",3105,43.7,47.1,2502,2343,618,922,46,54,1,15,1400,1178,5.7,42083
4,"Census Tract 21.02, Calhoun County, Alabama",3350,39.4,34.8,2576,2320,80,382,521,629,1,15,2102,1312,6.5,69875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54445,"Census Tract 14, Winnebago County, Wisconsin",3928,41.6,44.2,3661,2864,232,1234,326,123,55,139,1400,1827,5.4,47156
54446,"Census Tract 34, Winnebago County, Wisconsin",4606,34.5,35.2,3913,2987,211,1277,426,135,55,139,3400,1819,5.2,47628
54447,"Census Tract 11, Winnebago County, Wisconsin",4200,37.0,43.4,3687,2871,162,1140,458,153,55,139,1100,1990,4.7,35446
54448,"Census Tract 20, Winnebago County, Wisconsin",7192,41.8,42.4,7091,4909,220,1715,1080,338,55,139,2000,2886,6.0,68457


In [4]:
#Some of the data needs a geoid
#GEOID is defined as: STATE+COUNTY+TRACT
#See: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
def create_geoid(row):
    geoid = str(utils.pad_with_zero(row["state"])) + str(utils.pad_place_with_zero(row["county"])) + str(utils.pad_tract_with_zero(row["tract"]))
    return int(geoid)

census_tracts["GEOID"] = census_tracts.apply(create_geoid, axis=1)

### Other data files

In [5]:
census_tracts.shape

(54450, 17)

In [6]:
land_area = pd.read_csv("data/2019_Gaz_tracts_national.txt", sep="\t")
land_area

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,AL,1001020200,3325680,5669,1.284,0.002,32.475758,-86.472468
2,AL,1001020300,5349273,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6384276,8408,2.465,0.003,32.471030,-86.444835
4,AL,1001020500,11408866,43534,4.405,0.017,32.458922,-86.421826
...,...,...,...,...,...,...,...,...
73996,PR,72153750501,1820185,0,0.703,0.000,18.031211,-66.867347
73997,PR,72153750502,689930,0,0.266,0.000,18.024746,-66.860442
73998,PR,72153750503,3298433,1952,1.274,0.001,18.023148,-66.876603
73999,PR,72153750601,10987037,4527,4.242,0.002,18.017809,-66.839070


In [7]:
census_tracts["GEOID"]

0         1015002000
1         1015000800
2         1015000900
3         1015001400
4         1015002102
            ...     
54445    55139001400
54446    55139003400
54447    55139001100
54448    55139002000
54449    55139002100
Name: GEOID, Length: 54450, dtype: int64

In [8]:
tracts_with_land = census_tracts.merge(land_area[["GEOID","ALAND", "ALAND_SQMI"]], how = "left", on = "GEOID")
tracts_with_land.shape

(54450, 19)

In [9]:
#This is by block group so may need to align on whether to average or not?
epa = pd.read_csv("data/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")
epa

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07


In [10]:
epa["TRACTCE"].value_counts()

950100    1199
950200    1164
950300    1107
200       1049
300       1042
          ... 
864307       1
45116        1
188700       1
16712        1
212305       1
Name: TRACTCE, Length: 23969, dtype: int64

In [11]:
#EPA only has the 10 & 20 digit geoids and I want an 11 digit geoid
#Let's use the same process as earlier in the notebook
#Copying the comments from there below here too:

#Some of the data needs a geoid
#GEOID is defined as: STATE+COUNTY+TRACT
#See: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

def create_geoid_epa(row):
    geoid = str(utils.pad_with_zero(row["STATEFP"])) + str(utils.pad_place_with_zero(row["COUNTYFP"])) + str(utils.pad_tract_with_zero(row["TRACTCE"]))
    return int(geoid)

epa["GEOID"] = epa.apply(create_geoid_epa, axis=1)
epa

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,GEOID
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05,48113007825
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05,48113007825
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05,48113007825
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05,48113007824
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05,48113007824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05,78030961000
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05,78030961000
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05,78030961000
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07,78030960700


In [12]:
#Merge epa with census_tracts
#TODO: How do we want to define walkability? It's currently defined by blockgroup? Do we find average?
tracts_with_epa= census_tracts.merge(epa[["GEOID","NatWalkInd"]], how = "inner", on = "GEOID")
print("After merge, the shape is:", tracts_with_epa.shape)

# The shape includes additional rows since the epa data is provided by census block group (there are sometimes a few
# block groups within a tract)

# If we groupby GEOID after the merge and then take the mean of the walkability index, that should give us a column
tracts_with_epa.groupby("GEOID")["NatWalkInd"].agg("mean")

After merge, the shape is: (159160, 18)


GEOID
1015000200      8.250000
1015000300      9.750000
1015000400      5.277778
1015000500      7.833333
1015000600      6.416667
                 ...    
55139003400    11.625000
55139003500     8.000000
55139003600     8.611111
55139003701     7.944444
55139003702     6.472222
Name: NatWalkInd, Length: 54450, dtype: float64

In [13]:
#Let's update the tracts_with_land df with this new index
tracts_with_land_walk = tracts_with_land.merge(tracts_with_epa.groupby("GEOID")["NatWalkInd"].agg("mean"), on="GEOID")
tracts_with_land_walk

,NAME,B01003_001E,B01002_002E,B01002_001E,B02001_002E,B06009_001E,B06009_002E,B06009_003E,B06009_005E,B06009_006E,state,county,tract,DP02_0001E,DP04_0037E,DP03_0062E,GEOID,ALAND,ALAND_SQMI,NatWalkInd
0,"Census Tract 20, Calhoun County, Alabama",7331,39.2,39.8,6926,5131,387,1758,598,481,1,15,2000,2735,6.0,66217,1015002000,239320450,92.402,3.066667
1,"Census Tract 8, Calhoun County, Alabama",981,39.2,41.7,419,752,204,278,43,35,1,15,800,423,5.1,27656,1015000800,3607490,1.393,11.500000
2,"Census Tract 9, Calhoun County, Alabama",3617,40.9,44.3,2476,2610,266,638,459,448,1,15,900,1528,6.4,50810,1015000900,12114697,4.678,5.166667
3,"Census Tract 14, Calhoun County, Alabama",3105,43.7,47.1,2502,2343,618,922,46,54,1,15,1400,1178,5.7,42083,1015001400,21093280,8.144,5.291667
4,"Census Tract 21.02, Calhoun County, Alabama",3350,39.4,34.8,2576,2320,80,382,521,629,1,15,2102,1312,6.5,69875,1015002102,20789534,8.027,7.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54445,"Census Tract 14, Winnebago County, Wisconsin",3928,41.6,44.2,3661,2864,232,1234,326,123,55,139,1400,1827,5.4,47156,55139001400,6286797,2.427,12.583333
54446,"Census Tract 34, Winnebago County, Wisconsin",4606,34.5,35.2,3913,2987,211,1277,426,135,55,139,3400,1819,5.2,47628,55139003400,3913783,1.511,11.625000
54447,"Census Tract 11, Winnebago County, Wisconsin",4200,37.0,43.4,3687,2871,162,1140,458,153,55,139,1100,1990,4.7,35446,55139001100,2698989,1.042,14.666667
54448,"Census Tract 20, Winnebago County, Wisconsin",7192,41.8,42.4,7091,4909,220,1715,1080,338,55,139,2000,2886,6.0,68457,55139002000,182238158,70.363,6.100000


In [14]:
schools = pd.read_excel("data/GRF19/grf19_lea_tract.xlsx")
schools.rename(columns={"TRACT" : "GEOID"}, inplace=True)
schools[:10]

,LEAID,NAME_LEA19,GEOID,COUNT,LANDAREA,WATERAREA
0,100001,Fort Rucker School District,1031010300,2,23.428498,0.000000
1,100001,Fort Rucker School District,1045020000,2,66.513225,1.081745
2,100003,Maxwell AFB School District,1101000900,3,3.356590,0.143795
3,100003,Maxwell AFB School District,1101001000,3,0.001526,0.000000
4,100003,Maxwell AFB School District,1101006000,3,0.003588,0.000000
5,100005,Albertville City School District,1095030701,8,2.116012,0.000000
6,100005,Albertville City School District,1095030702,8,0.800889,0.009930
7,100005,Albertville City School District,1095030801,8,5.437714,0.012075
8,100005,Albertville City School District,1095030802,8,5.297315,0.017784
9,100005,Albertville City School District,1095030902,8,7.293396,0.033274


In [15]:
tracts_with_schools = tracts_with_land.merge(schools, how = "left", on = "GEOID")
tracts_with_schools.shape

(76380, 24)

In [16]:
#Some geoids map to multiple school districts so let's aggregate them all into a list
tracts_with_schools.groupby("GEOID")["NAME_LEA19"].agg(list)

GEOID
1015000200                       [Anniston City School District]
1015000300                       [Anniston City School District]
1015000400     [Anniston City School District, Calhoun County...
1015000500     [Anniston City School District, Calhoun County...
1015000600     [Anniston City School District, Calhoun County...
                                     ...                        
55139003400                             [Neenah School District]
55139003500                             [Neenah School District]
55139003600                             [Neenah School District]
55139003701                             [Neenah School District]
55139003702                             [Neenah School District]
Name: NAME_LEA19, Length: 54450, dtype: object

In [17]:
tracts_with_land_walk_schools = tracts_with_land_walk.merge(tracts_with_schools.groupby("GEOID")["NAME_LEA19"].agg(list), on="GEOID")
tracts_with_land_walk_schools

,NAME,B01003_001E,B01002_002E,B01002_001E,B02001_002E,B06009_001E,B06009_002E,B06009_003E,B06009_005E,B06009_006E,...,county,tract,DP02_0001E,DP04_0037E,DP03_0062E,GEOID,ALAND,ALAND_SQMI,NatWalkInd,NAME_LEA19
0,"Census Tract 20, Calhoun County, Alabama",7331,39.2,39.8,6926,5131,387,1758,598,481,...,15,2000,2735,6.0,66217,1015002000,239320450,92.402,3.066667,"[Calhoun County School District, Jacksonville ..."
1,"Census Tract 8, Calhoun County, Alabama",981,39.2,41.7,419,752,204,278,43,35,...,15,800,423,5.1,27656,1015000800,3607490,1.393,11.500000,[Anniston City School District]
2,"Census Tract 9, Calhoun County, Alabama",3617,40.9,44.3,2476,2610,266,638,459,448,...,15,900,1528,6.4,50810,1015000900,12114697,4.678,5.166667,"[Anniston City School District, Calhoun County..."
3,"Census Tract 14, Calhoun County, Alabama",3105,43.7,47.1,2502,2343,618,922,46,54,...,15,1400,1178,5.7,42083,1015001400,21093280,8.144,5.291667,[Calhoun County School District]
4,"Census Tract 21.02, Calhoun County, Alabama",3350,39.4,34.8,2576,2320,80,382,521,629,...,15,2102,1312,6.5,69875,1015002102,20789534,8.027,7.166667,"[Calhoun County School District, Jacksonville ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54445,"Census Tract 14, Winnebago County, Wisconsin",3928,41.6,44.2,3661,2864,232,1234,326,123,...,139,1400,1827,5.4,47156,55139001400,6286797,2.427,12.583333,[Oshkosh Area School District]
54446,"Census Tract 34, Winnebago County, Wisconsin",4606,34.5,35.2,3913,2987,211,1277,426,135,...,139,3400,1819,5.2,47628,55139003400,3913783,1.511,11.625000,[Neenah School District]
54447,"Census Tract 11, Winnebago County, Wisconsin",4200,37.0,43.4,3687,2871,162,1140,458,153,...,139,1100,1990,4.7,35446,55139001100,2698989,1.042,14.666667,[Oshkosh Area School District]
54448,"Census Tract 20, Winnebago County, Wisconsin",7192,41.8,42.4,7091,4909,220,1715,1080,338,...,139,2000,2886,6.0,68457,55139002000,182238158,70.363,6.100000,"[Omro School District, Oshkosh Area School Dis..."


In [18]:
tracts_with_land_walk_schools.to_csv(f"{data_directory}/merged_data.csv", index=False)

## Next let's manipulate some of the variables to create new columns to be what we want to compare

In [27]:
# Let's rename our big df
#Indices to keep as is: NatWalkInd, ALAND, AWATER, 
df = pd.read_csv(f"{data_directory}/merged_data.csv")
new_columns = ["population_density", "housing_density", "median_age", "median_male_age", "pct_white", "median_income",
              "school_districts", "pct_lt_hs_grad", "median_rooms", "pct_hs_grad", "pct_at_least_col_grad", "GEOID",
               "state", "tract", "NAME", "NatWalkInd", "ALAND", "ALAND_SQMI", "county"]

In [28]:
df.columns

Index(['NAME', 'B01003_001E', 'B01002_002E', 'B01002_001E', 'B02001_002E',
       'B06009_001E', 'B06009_002E', 'B06009_003E', 'B06009_005E',
       'B06009_006E', 'state', 'county', 'tract', 'DP02_0001E', 'DP04_0037E',
       'DP03_0062E', 'GEOID', 'ALAND', 'ALAND_SQMI', 'NatWalkInd',
       'NAME_LEA19'],
      dtype='object')

In [29]:
#Columns to be renamed
df = df.rename(columns={"B01002_001E" : "median_age" , "B01002_002E" : "median_male_age" ,
                    "DP03_0062E" : "median_income",  "NAME_LEA19" : "school_districts" , 
                        "DP04_0037E" : "median_rooms"})

In [30]:
def safe_division(x, y):
    try:
        return x / y
    except:
        return 0

In [31]:
#Columns to be created
df["population_density"] = df.apply(lambda x: safe_division(float(x['B01003_001E']),float(x["ALAND_SQMI"])), axis=1)
df["housing_density"] = df.apply(lambda x: safe_division(float(x['DP02_0001E']),float(x["ALAND_SQMI"])), axis=1)
df["pct_white"] = df.apply(lambda x: safe_division(float(x['B02001_002E']),float(x["B01003_001E"])), axis=1)
df["pct_lt_hs_grad"] = df.apply(lambda x: safe_division(float(x['B06009_002E']),float(x["B06009_001E"])), axis=1)
df["pct_hs_grad"] = df.apply(lambda x: safe_division(float(x['B06009_003E']),float(x["B06009_001E"])), axis=1)
df["pct_at_least_col_grad"] = df.apply(lambda x: safe_division((float(x["B06009_005E"]) + float(x["B06009_006E"])),float(x["B06009_001E"])), axis=1)

In [32]:
df.columns

Index(['NAME', 'B01003_001E', 'median_male_age', 'median_age', 'B02001_002E',
       'B06009_001E', 'B06009_002E', 'B06009_003E', 'B06009_005E',
       'B06009_006E', 'state', 'county', 'tract', 'DP02_0001E', 'median_rooms',
       'median_income', 'GEOID', 'ALAND', 'ALAND_SQMI', 'NatWalkInd',
       'school_districts', 'population_density', 'housing_density',
       'pct_white', 'pct_lt_hs_grad', 'pct_hs_grad', 'pct_at_least_col_grad'],
      dtype='object')

In [33]:
new_df = df[new_columns]
new_df.columns

Index(['population_density', 'housing_density', 'median_age',
       'median_male_age', 'pct_white', 'median_income', 'school_districts',
       'pct_lt_hs_grad', 'median_rooms', 'pct_hs_grad',
       'pct_at_least_col_grad', 'GEOID', 'state', 'tract', 'NAME',
       'NatWalkInd', 'ALAND', 'ALAND_SQMI', 'county'],
      dtype='object')

In [34]:
new_df.to_csv(f"{data_directory}/tracts_data_v1.csv", index=False)

## Random Explorations- currently unused

In [3]:
df = pd.read_csv(f"{data_directory}/tracts_data_v1.csv")

In [20]:
tracts_to_cities = pd.read_csv(f"{data_directory}/tracts_to_cities.csv")
utils.pad_tract_with_zero(tracts_to_cities["tract"])


'0          201.0\n1          201.0\n2          202.0\n3          203.0\n4          203.0\n           ...  \n123628    9511.0\n123629    9511.0\n123630    9511.0\n123631    9513.0\n123632    9513.0\nName: tract, Length: 123633, dtype: float64'

In [17]:
tracts_with_cities = df.merge(tracts_to_cities[["tract","placenm"]], how = "left", on = "tract")
tracts_with_cities.shape

/Users/joanieweaver/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1151: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


(349868, 19)

In [7]:
def create_geoid(row):
    geoid = str(utils.pad_with_zero(row["state"])) + str(utils.pad_place_with_zero(row["county"])) + str(utils.pad_tract_with_zero(row["tract"]))
    return int(geoid)

tracts_to_cities["GEOID"] = tracts_to_cities.apply(create_geoid, axis=1)
tracts_to_cities

ValueError: invalid literal for int() with base 10: '0110010201.0'